# Use dropbox for persistent storage

Code taken from the dropbox [updown.py](https://github.com/dropbox/dropbox-sdk-python/blob/master/example/updown.py) example.

Follow the [instructions](https://github.com/dropbox/dropbox-sdk-python#creating-an-application) to create a *Dropbox Application* and to *Obtain an Access Token*. Store it in `DROPBOX_APIkey`.

Use `list_folder`, `download` and `upload` functions to store and retrive content in json format.

Documentation of the dropbox Python API is [here](https://dropbox-sdk-python.readthedocs.io/en/stable/).

In [ ]:
import os
import json
import dropbox

# DROPBOX_APIkey = '<your key>'
dbx = dropbox.Dropbox(DROPBOX_APIkey)

In [ ]:
def storage_download(fname):
    """Download a file.
    Return the bytes of the file, or None if it doesn't exist.
    """
    if not fname.startswith('/'):
        fname = '/{}'.format(fname)
    try:
        md, res = dbx.files_download(fname)
    except dropbox.exceptions.HttpError as err:
        print('*** HTTP error', err)
        return None
    except dropbox.exceptions.ApiError as err:
        # print('*** API error', err)
        return None
    data = res.content
    return data


def storage_upload(data, fname, overwrite=True):
    """Upload a file.
    Return the request response, or None in case of error.
    """
    if not fname.startswith('/'):
        fname = '/{}'.format(fname)

    if overwrite:
        mode = dropbox.files.WriteMode.overwrite
    else:
        mode = dropbox.files.WriteMode.add

    try:
        res = dbx.files_upload(
        data.encode('UTF8'), fname, mode, mute=True)
    except dropbox.exceptions.ApiError as err:
        print('*** API error', err)
        return None
    return res


def storage_list(folder=''):
    """List a folder.
    Return a dict mapping unicode filenames to
    FileMetadata|FolderMetadata entries.
    """
    path = '/{}'.format(folder)
    path = path.rstrip('/')
    try:
        res = dbx.files_list_folder(path)
    except dropbox.exceptions.ApiError as err:
        print('Folder listing failed for', path, '-- assumed empty:', err)
        return {}
    else:
        rv = {}
        for entry in res.entries:
            rv[entry.name] = entry
        return rv

In [ ]:
files = storage_list('')
print('Files in storage, top level ({}): {}{}'.format(len(files.keys()), 
                                                      ', '.join(sorted(files.keys())[:5]),
                                                      ', ...' if len(files) > 5 else ''))

In [ ]:
res = storage_upload('test content', 'test/test.txt')
storage_download('test/test.txt')